# Experiments expanding on Ito et al. (2017)


Set up environment

In [ ]:
import copy
import sys
sys.path.append('../utils/')
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
import pandas as pd
import seaborn as sns
sns.set_style("white")
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA, ARMA
from statsmodels.tsa.filters.filtertools import recursive_filter

## Import custom modules
# Primary module with most model functions
import model
# Module for FC regression
import multregressionconnectivity as mreg

Helper functions

In [ ]:
phi = lambda x: np.tanh(x)

def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)

def scatter_two_vars(a,b):
    d = {'Var1': a, 'Var2': b}
    df = pd.DataFrame(data=d)
    p = sns.regplot(x='Var1', y='Var2', data = df)
    p.text(min(a), max(b), "r = %s"%(str(round(np.corrcoef(a, b)[1][0],3))), horizontalalignment='left', color='black')

## Experiment 1: GLM recovery of simulated task activity


### Create network

One hub and two local networks

In [ ]:
#### Set up subject network matrix ####
# Parameters for subject's networks
ncommunities = 3
innetwork_dsity = .60
outnetwork_dsity = .08
hubnetwork_dsity = .25

nodespercommunity = 35
totalnodes = nodespercommunity*ncommunities

# Construct structural matrix
W = model.generateStructuralNetwork(ncommunities=ncommunities,
                                    innetwork_dsity=innetwork_dsity,
                                    outnetwork_dsity=outnetwork_dsity,
                                    hubnetwork_dsity=hubnetwork_dsity,
                                    nodespercommunity=nodespercommunity,
                                    showplot=False)
# Construct synaptic matrix
G = model.generateSynapticNetwork(W, showplot=True)

#### Simulate resting state activity using information flow model

Apply neural networl]k model **(Eq 3 in paper)** to the connectivity matrix and calculate each nodes' activity for each time point

**Note that this is still in synaptic space (i.e. not convolved)**

In [ ]:
## Set up simulation parameters
#Tmaxrest = 60000 # 1min resting state data
Tmaxrest = 300000 # 5min resting state data
dt = 1.0
tau = 1.0
g = 1.0
s = 1.0
TRLength = 100

restdata = model.networkModel(G, 
                              Tmax=Tmaxrest,
                              dt=dt,
                              g=g,
                              s=s,
                              tau=tau,
                              I=None,
                              noise = 1)

In [ ]:
plt.figure()
plt.imshow(restdata, aspect='auto',origin='lower')
plt.title('Simulated resting-state activity', y=1.04, fontsize=16)
plt.ylabel('Regions',fontsize=14)
plt.xlabel('Time (s)',fontsize=14)
plt.xticks(np.arange(0,Tmaxrest+1,10000),np.arange(0,Tmaxrest+1,10000)/100)
plt.colorbar()

#### Convolve resting state activity

In [ ]:
restfmri = model.convolveTimeseries(restdata,
                                    samplingrate=dt,
                                    TRLength=TRLength)

In [ ]:
plt.figure()
plt.imshow(restfmri, aspect='auto',origin='lower')
plt.title('Simulated fMRI resting-state activity', y=1.04, fontsize=16)
plt.ylabel('Regions',fontsize=14)
plt.xlabel('Time (s)',fontsize=14)
plt.colorbar()

#### Recover connectivity matrix from resting state activity

In [ ]:
fcmat = np.corrcoef(restfmri)
# 0 out the diagonal
np.fill_diagonal(fcmat,0)

plt.figure()
plt.imshow(fcmat, origin='lower')
plt.colorbar()
plt.title('Simulated fMRI resting-state FC \nSingle Subject\nPearsonFC', y=1.04, fontsize=16)
plt.xlabel('Regions',fontsize=14)
plt.ylabel('Regions', fontsize=14)
plt.tight_layout()

In [ ]:
fcmat_mreg = mreg.multregressionconnectivity(restfmri)
# 0 out the diagonal
np.fill_diagonal(fcmat_mreg,0)

plt.figure()
plt.imshow(fcmat_mreg, origin='lower')
plt.colorbar()
plt.title('Simulated fMRI resting-state FC\nSingle Subject\nMultipleRegressionFC', y=1.04, fontsize=16)
plt.xlabel('Regions',fontsize=14)
plt.ylabel('Regions', fontsize=14)
plt.tight_layout()

How **quantitatively** similar is the fc_mat to the weight matrix G?

In [ ]:
plt.scatter(fcmat.flatten(), G.flatten())
np.corrcoef(fcmat.flatten(), G.flatten())

In [ ]:
plt.scatter(fcmat_mreg.flatten(), G.flatten())
np.corrcoef(fcmat_mreg.flatten(), G.flatten())

### Simulate task activity using resting state

- How do you decide the task's spatial specificity, i.e. which network and nodes within a network it affects primarily? 
    - In the simulations the approach is: For topdown tasks only the hub network is simulated. E.g. for task 1 the first quarter of nodes are stimulated, for task 2 the next quarter etc. 
    - For topdown and bottom up (where a bug was fixed) it is a bit more complicated. E.g. for task 5 the first quarter of the hub network and the nodes of network 1 (first local network) that connect to the hub network (network 0) are stimulated

#### Hub network stimulation

In [ ]:
Ci = np.repeat(np.arange(ncommunities),nodespercommunity) # Construct a community affiliation vector
hub_ind = np.where(Ci==0)[0] # Identify the regions associated with the hub network (hub network is by default the 0th network)

# Set number of time points for each task
#Tmax = 10000
Tmax = 300000 #increased to 5 minutes

T = np.arange(0,Tmax,dt)

taskdata = {}
stimtimes = {}
    
# Construct timing array for convolution -- this timing is irrespective of the task being performed
# Tasks are only determined by which nodes are stimulated
tasktiming = np.zeros((1,len(T)))
for t in range(len(T)):
    if t%2000>500 and t%2000<1000:
    #if t>0 and t%1000==0: #Changed to a task with spikes as activity
        tasktiming[0,t] = 1.0

In [ ]:
topdown_only = range(1,2)
task = 1

taskcount = task-np.min(topdown_only)
stimsize = np.floor(nodespercommunity/4.0)
stim_nodes = np.arange((taskcount)*stimsize,(taskcount)*stimsize+stimsize,dtype=int)
stimtimes[task] = np.zeros((totalnodes,len(T)))

for t in range(len(T)):
    if tasktiming[0,t] == 1:
        # Task stimulation every 10 seconds for 4 seconds, excluding the first 10 seconds
        # Changed to spike every second for 10 minues
        stimtimes[task][stim_nodes,t] = .5

Block design ~~Spike task~~ looks like:

In [ ]:
plt.plot(T[:10000], tasktiming[0,:10000])
plt.ylim(top = 1.2, bottom = -0.1)

Activated nodes in the hub network *only* (making it a top-down task) are:

In [ ]:
stim_nodes

In [ ]:
taskdata[task] = model.networkModel(G,Tmax=Tmax,dt=dt,g=g,s=s,tau=tau,
                                          I=stimtimes[task], noise=1)

In [ ]:
plt.figure()
plt.imshow(taskdata[task], aspect='auto',origin='lower')
plt.title('Simulated single topdown task activity', y=1.04, fontsize=16)
plt.ylabel('Regions',fontsize=14)
plt.xlabel('Time (s)',fontsize=14)
plt.xticks(np.arange(0,Tmax+1,10000),np.arange(0,Tmax+1,10000)/100)
plt.colorbar()

##### Convolve task activity 


In [ ]:
samplingrate=1.0
# Convolve simulated task data into fMRI signal
task_bold = model.convolveTimeseries(taskdata[task],
                               samplingrate=samplingrate, 
                               TRLength=TRLength)


In [ ]:
plt.figure()
plt.imshow(task_bold, aspect='auto',origin='lower')
plt.title('Simulated single topdown task fMRI activity', y=1.04, fontsize=16)
plt.ylabel('Regions',fontsize=14)
plt.xlabel('Time (s)',fontsize=14)
plt.colorbar()

##### Run GLM on convolved task activity

In [ ]:
n_skip_BOLD = int(TRLength)
# Temporally downsample task timing to seconds (before convolving)
timingconv = tasktiming[:,::n_skip_BOLD]
timingconv.shape = (timingconv.shape[1],)
hrfsample_rate=1.0 # HRF is sampled at seconds
hrfsample_times = np.arange(0, 30, hrfsample_rate, dtype=float)
hrf_at_simsample = model.hrf(hrfsample_times)
hrfconvtime = np.convolve(timingconv, hrf_at_simsample)
n_to_remove = len(hrf_at_simsample) - 1
convolved = hrfconvtime[:-n_to_remove]
# Output
timing_convolved_downsampled = convolved

In [ ]:
task_betas_postfmri, task_resids_postfmri, task_tvals_postfmri, task_betas_ci_postfmri = model.runTaskGLM(task_bold, timing_convolved_downsampled)

Results of GLM. Vertical lines indicate network boundaries. Stimulated nodes are the first eight in the hub network.

In [ ]:
plt.plot(task_tvals_postfmri)
plt.ylabel('Uncorrected t-value',fontsize=14)
plt.xlabel('Node',fontsize=14)
plt.axhline(y=3,linewidth=2, color='red', ls = "--")
plt.axvline(x=35,linewidth=2, color='black', ls = "--")
plt.axvline(x=70,linewidth=2, color='black', ls = "--")

In [ ]:
uperror = task_betas_ci_postfmri[:,1]
downerror = task_betas_ci_postfmri[:,0]
errorbars = np.concatenate(([uperror], [downerror]), axis=0)

plt.errorbar(np.arange(totalnodes), task_betas_postfmri, yerr=errorbars, fmt = 'o')
plt.ylabel('Beta (with 95% CI)',fontsize=14)
plt.xlabel('Node',fontsize=14)

In [ ]:
hub_stim_nodes = stim_nodes

### Relationship between betas and weights from the connectivity matrix

Extract average connectivity between each node and the nodes stimulated by task.  
Correlate these average weights with betas for each node.

NOTE: The weight matrix is *NOT* symmetric (I think) because afferent and efferent connection weights need not be the same.

In [ ]:
check_symmetric(G)

Extract average connectivity of each node with the stim nodes

In [ ]:
afferent_cons = np.empty((0,G.shape[0]), float)
efferent_cons = np.empty((0,G.shape[0]), float)
for i in range(G.shape[0]):
    efferent_cons = np.append(efferent_cons, [G[hub_stim_nodes,i].mean()])
    afferent_cons = np.append(afferent_cons, [G[i,hub_stim_nodes].mean()])

In [ ]:
d = {'Beta': task_betas_postfmri[8:], 'Connectivity': efferent_cons[8:]}
df = pd.DataFrame(data=d)
p = sns.regplot(x='Beta', y='Connectivity', data = df)
p.text(min(task_betas_postfmri[8:]), max(efferent_cons[8:]), "r = %s"%(str(round(np.corrcoef(task_betas_postfmri[8:], efferent_cons[8:])[1][0],3))), horizontalalignment='left', color='black')
p.set_title('Correlation between betas and efferent connections')

In [ ]:
d = {'Beta': task_betas_postfmri[8:], 'Connectivity': afferent_cons[8:]}
df = pd.DataFrame(data=d)
p = sns.regplot(x='Beta', y='Connectivity', data = df)
p.text(min(task_betas_postfmri[8:]), max(afferent_cons[8:]), "r = %s"%(str(round(np.corrcoef(task_betas_postfmri[8:], afferent_cons[8:])[1][0],3))), horizontalalignment='left', color='black')
p.set_title('Correlation between betas and afferent connections')

## Correcting GLM using whole system data

### Uncorrected GLM at neural network level

Using first third of the data for speed of estimation

In [ ]:
taskdata_short = copy.copy(taskdata[1][:,:10000])

In [ ]:
def pshift(y,p):
            y_lag = np.roll(y, p)
            y_lag[:p] = 0
            return y_lag

def recover_pars(x, y, p = 1):
        
    ntimepoints = y.shape[1]
    nregions = y.shape[0]
    
    #Initialize empty variables
    task_glm = np.zeros((nregions))
    cur_node_ext_ucr = np.zeros((nregions)) 
    weight_matrix_ext_ucr = np.zeros((nregions, nregions-1))
    task_ext_ucr = np.zeros((nregions))
    cur_node_ext_arc = np.zeros((nregions))
    weight_matrix_ext_arc = np.zeros((nregions, nregions-1))
    task_ext_arc = np.zeros((nregions))
    cur_node_ext_pw = np.zeros((nregions))
    weight_matrix_ext_pw = np.zeros((nregions, nregions-1))
    task_ext_pw = np.zeros((nregions))

    #design matrix/task regressor
    task_reg = x.T
    
    for region in range(0, nregions):
    
    #STANDARD GLM  
        cur_y = y[region,:]
        mod_glm = sm.OLS(cur_y, task_reg)
        res_glm = mod_glm.fit()
        task_glm[regions] = res_glm.params[0]
        
    #EXTENDED GLM
    #UCR pars
        
        # New y: Shift time course up and drop last time point to model activity in next time step
        cur_yp1 = np.roll(cur_y, -1)[:-1]
        
        #Make design matrix
        # Add current nodes activity in current time step dropping last time step
        cur_node = cur_y[:-1].reshape(-1,1)
        cur_node_t = np.apply_along_axis(phi, 0, cur_node)
        
        # Add other nodes' activity in current time step dropping last time step
        drop_region = [region]
        other_nodes = np.delete(taskdata_short_pw, drop_region, axis=0)
        other_nodes = other_nodes[:,:-1]
        other_nodes = other_nodes.T
        other_nodes_t = np.apply_along_axis(phi, 0, other_nodes)
        
        ucr_des_mat = np.concatenate((cur_node_t, other_nodes_t), 1)
        
        # Add task activity in current time step
        ucr_des_mat = np.concatenate((ucr_des_mat, task_reg[:-1,:]), 1)
        
        #Run model and extract parameters
        mod_ext_ucr = sm.OLS(cur_yp1, ucr_des_mat)
        res_ext_ucr = mod_ext_ucr.fit()
        cur_node_ext_ucr[region] = res_ext_ucr.params[0]
        weight_matrix_ext_ucr[region:] = res_ext_ucr.params[1:-1]
        task_ext_ucr[region] = res_ext_ucr.params[-1]
    
    #ARC pars
        # Add untransformed activity in current node for current time step
        arc_des_mat = np.concatenate((ucr_des_mat, cur_node), 1)
        
        # Add transformed activity in current node from previous time step
        cur_node_m1 = np.roll(cur_node, 1)
        cur_node_m1 = np.insert(cur_node, 0, 0)
        cur_node_m1 = cur_node_m1.reshape(-1,1)
        cur_node_m1_t = np.apply_along_axis(phi, 0, cur_node_m1)
        arc_des_mat = np.concatenate((arc_des_mat, cur_node_m1_t), 1)
        
        # Add transformed activity from other nodes from previous time step
        other_nodes_m1 = np.roll(other_nodes, 1)
        other_nodes_m1 = np.insert(other_nodes, 0, 0, axis=0)
        other_nodes_m1 = other_nodes_m1.T
        other_nodes_m1_t = np.apply_along_axis(phi, 0, other_nodes_m1)
        arc_des_mat = np.concatenate((arc_des_mat, other_nodes_m1_t), 1)
        
        # Add task activity from previous time step
        task_reg_m1 = np.roll(task_reg[:-1,:], 1)
        task_reg_m1 = np.insert(task_reg_m1, 0, 0)
        
        arc_des_mat = np.concatenate((ucr_des_mat, task_reg_m1), 1)
        
        #Run model and extract parameters
        mod_ext_arc = sm.OLS(cur_yp1, arc_des_mat)
        res_ext_arc = mod_ext_arc.fit()
        cur_node_ext_arc[region] = res_ext_arc.params[0]
        weight_matrix_ext_arc[region:] = res_ext_arc.params[1:-1]
        task_ext_ucr[region] = res_ext_arc.params[-1]
        
    #PW pars
    
        # Extract errors from res_ext_ucr
        err_ext_ucr = res_ext_ucr.resid
        
        # Run AR(p) model on residuals
        try:
            ar_mod = ARMA(err_ext_ucr, order=(p, 0))
            ar_res = ar_mod.fit(trend="nc")
            
        # Prewhiten cur_yp1 and ucr_des_mat
            shift_y = pshift(cur_yp1, p)
            shift_X = pshift(ucr_des_mat, p)
            mpw_y = cur_yp1 - (ar_res.params*shift_y)
            mpw_X = ucr_des_mat - (ar_res.params*shift_X)
            mod_ext_pw = sm.OLS(mpw_y, mpw_X)
            res_ext_pw = mod_ext_pw.fit()
            cur_node_ext_pw[region] = res_ext_pw.params[0]
            weight_matrix_ext_pw[region:] = res_ext_pw.params[1:-1]
            task_ext_pw[region] = res_ext_pw.params[-1]
        
        except:
            print('Whitening failed!')
            cur_node_ext_pw[region] = np.nan
            weight_matrix_ext_pw[region:] = np.nan
            task_ext_pw[region] = np.nan
    
    
    out = {'task_glm':task_glm,
           'cur_node_ext_ucr':cur_node_ext_ucr,
           'weight_matrix_ext_ucr':weight_matrix_ext_ucr,
           'task_ext_ucr': task_ext_ucr,
           'cur_node_ext_arc':cur_node_ext_arc,
           'weight_matrix_ext_arc':weight_matrix_ext_arc,
           'task_ext_arc':task_ext_arc,
           'cur_node_ext_pw':cur_node_ext_pw,
           'weight_matrix_ext_pw':weight_matrix_ext_pw,
           'task_ext_pw':task_ext_pw}
    
    return (out)

### Corrected GLM at neural network level

#### Task betas reduced after correction?


#### Current node betas == 1?

#### Other node betas recapitulate connectivity matrix?

**Relationship between other nodes betas and the afferent connectivity weights (can you recover the connectivity matrix?)**